In [120]:
from sedona.spark import *
from pathlib import Path
from pyspark.sql.functions import col, lower
from pyspark.sql import DataFrame
import json

from ipyleaflet import Map, basemaps, basemap_to_tiles, MarkerCluster, Marker, AwesomeIcon
from ipywidgets import Layout
import numpy as np

In [2]:
# get the project root dir
project_root_dir = Path.cwd().parent.parent
data_dir = f"{project_root_dir}/data"
win_root_dir = "C:/Users/PLIU/Documents/ubuntu_share/data_set"
fr_commune_file_path = f"{win_root_dir}/kaggle/geospatial/communes_fr_geoparquet"

In [3]:
# build a sedona session (sedona = 1.6.1)
jar_folder = Path(f"{project_root_dir}/jars/sedona-35-213-161")
jar_list = [str(jar) for jar in jar_folder.iterdir() if jar.is_file()]
jar_path = ",".join(jar_list)

# build a sedona session (sedona = 1.6.1) offline
config = SedonaContext.builder() \
    .master("local[*]") \
    .config('spark.jars', jar_path). \
    getOrCreate()

In [4]:
# create a sedona context
sedona = SedonaContext.create(config)
sc = sedona.sparkContext
spark = sedona.getActiveSession()


In [5]:
# this sets the encoding of shape files
sc.setSystemProperty("sedona.global.charset", "utf8")

In [6]:
fr_commune_df = sedona.read.format("geoparquet").load(fr_commune_file_path)

In [7]:
fr_commune_df.show(5)
fr_commune_df.printSchema()

+--------------------+--------------------+--------------------+------------+-----+
|            geometry|           wikipedia|             surf_ha|         nom|insee|
+--------------------+--------------------+--------------------+------------+-----+
|POLYGON ((9.32016...|fr:Pie-d'Orezza  ...|     573.00000000...|Pie-d'Orezza|2B222|
|POLYGON ((9.20010...|fr:Lano          ...|     824.00000000...|        Lano|2B137|
|POLYGON ((9.27757...|fr:Cambia        ...|     833.00000000...|      Cambia|2B051|
|POLYGON ((9.25119...|fr:Érone         ...|     393.00000000...|       Érone|2B106|
|POLYGON ((9.28339...|fr:Oletta        ...|    2674.00000000...|      Oletta|2B185|
+--------------------+--------------------+--------------------+------------+-----+
only showing top 5 rows

root
 |-- geometry: geometry (nullable = true)
 |-- wikipedia: string (nullable = true)
 |-- surf_ha: string (nullable = true)
 |-- nom: string (nullable = true)
 |-- insee: string (nullable = true)



In [121]:
fr_commune_df = fr_commune_df.select("geometry","nom","insee").withColumn("name",lower(col("nom"))).drop("nom")

In [122]:
fr_commune_df.show(5)
fr_commune_df.printSchema()

+--------------------+-----+------------+
|            geometry|insee|        name|
+--------------------+-----+------------+
|POLYGON ((9.32016...|2B222|pie-d'orezza|
|POLYGON ((9.20010...|2B137|        lano|
|POLYGON ((9.27757...|2B051|      cambia|
|POLYGON ((9.25119...|2B106|       érone|
|POLYGON ((9.28339...|2B185|      oletta|
+--------------------+-----+------------+
only showing top 5 rows

root
 |-- geometry: geometry (nullable = true)
 |-- insee: string (nullable = true)
 |-- name: string (nullable = true)



In [131]:
temp_table_name="fr_commune"
fr_commune_df.createOrReplaceTempView(temp_table_name)

In [12]:
def get_nearest_commune(latitude:str, longitude:str, max_commune_number:int):
    nearest_commune_df = sedona.sql(f"""
     SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('{longitude},{latitude}', ','), z.geometry) AS distance FROM {temp_table_name} as z ORDER BY distance ASC LIMIT {max_commune_number}
     """)
    return nearest_commune_df

In [13]:
# the gps coordinates for kremlin-Bicetre is 48.8100° N, 2.3539° E

kb_latitude = "48.8100"
kb_longitude = "2.3539"

In [16]:
kb_nearest_shape_df = get_nearest_commune(kb_latitude,kb_longitude,36000)

In [17]:
%%time

kb_nearest_shape_df.show(50)
kb_nearest_shape_df.count()

+--------------------+-----+------------------+
|        commune_name|insee|          distance|
+--------------------+-----+------------------+
|  Le Kremlin-Bicêtre|94043|198.60307108585405|
|            Gentilly|94037| 798.3521490770968|
|             Arcueil|94003|1543.0937442695515|
|           Villejuif|94076| 2007.793912679607|
|      Ivry-sur-Seine|94041| 2489.634383841373|
|              Cachan|94016| 2590.828517555236|
|           Montrouge|92049| 2750.714176859015|
|             Bagneux|92007| 3462.091511432535|
|     Vitry-sur-Seine|94081|3845.1624363327196|
|     L'Haÿ-les-Roses|94038| 3942.190017739479|
|   Charenton-le-Pont|94018|4109.1534066729655|
|      Bourg-la-Reine|92014| 4252.649916385617|
|            Malakoff|92046| 4299.536384480496|
|      Chevilly-Larue|94021| 4792.395404715219|
|           Châtillon|92020| 4852.777041114918|
|              Vanves|92075| 5035.982282488627|
|         Alfortville|94002|   5173.1066517449|
|               Paris|75056| 5246.572543

34955

In [18]:
# the gps coordinates for Paul-Brousse is 48.7951606539 N, 2.3636935981061606 E
pb_latitude = "48.7951606539"
pb_longitude = "2.3636935981061606"

In [21]:
pb_nearest_shape_df = get_nearest_commune(pb_latitude,pb_longitude,36000)

In [22]:
%%time

pb_nearest_shape_df.show(10)
pb_nearest_shape_df.count()

+------------------+-----+------------------+
|      commune_name|insee|          distance|
+------------------+-----+------------------+
|         Villejuif|94076| 417.1387367323394|
|Le Kremlin-Bicêtre|94043|1616.4950325552602|
|            Cachan|94016|2350.1367377095153|
|   Vitry-sur-Seine|94081| 2391.395059245885|
|           Arcueil|94003| 2455.207543731906|
|          Gentilly|94037| 2469.389110389481|
|    Ivry-sur-Seine|94041|2592.6844195169124|
|   L'Haÿ-les-Roses|94038| 2842.834896668716|
|    Chevilly-Larue|94021| 3239.030695241757|
|    Bourg-la-Reine|92014| 3788.693247539781|
+------------------+-----+------------------+
only showing top 10 rows

CPU times: total: 0 ns
Wall time: 2.27 s


34955

In [23]:
ile_france_pbf_path=f"{win_root_dir}/geo_spatial/ile-de-france-geo-parquet"
osm_ile_france_df = spark.read.parquet(ile_france_pbf_path)

In [25]:
osm_ile_france_df.show(15)

+------+----+------------------+------------------+-----+---------+--------------------+--------------------+
|    id|type|          latitude|         longitude|nodes|relations|                tags|                info|
+------+----+------------------+------------------+-----+---------+--------------------+--------------------+
|122626|   0|49.115966300000004|         2.5549119|   []|       []|                  {}|{3, 2020-05-10 11...|
|122627|   0|49.110294100000004|         2.5521725|   []|       []|                  {}|{4, 2009-02-13 19...|
|122631|   0|        49.0834393|2.5511375000000003|   []|       []|                  {}|{15, 2021-06-30 1...|
|122632|   0|        49.0675225|2.5524679000000003|   []|       []|                  {}|{17, 2019-04-10 1...|
|122633|   0|         49.063616|2.5522412000000005|   []|       []|                  {}|{17, 2009-02-13 1...|
|122634|   0|        49.0597465|2.5509097000000005|   []|       []|                  {}|{2, 2009-02-13 19...|
|122635|  

In [31]:
def show_row_details(entity_id:int):
    sample_row = osm_ile_france_df.filter(osm_ile_france_df.id == entity_id )
    sample_row.show(truncate=False, vertical=True)

In [28]:
hospital_df = osm_ile_france_df.select("id", "latitude", "longitude", "tags").where("element_at(tags, 'amenity') in ('hospital', 'clinic')")

In [34]:
doctor_df = osm_ile_france_df.select("id", "latitude", "longitude", "tags").where("element_at(tags, 'amenity') == 'doctors'")

In [29]:
hospital_df.count()

564

In [30]:
hospital_df.show(5)

+---------+------------------+------------------+--------------------+
|       id|          latitude|         longitude|                tags|
+---------+------------------+------------------+--------------------+
|452401907| 48.78635369999983| 2.291711499999987|{amenity -> hospi...|
|476313165|48.878517699999826| 2.414601899999999|{name -> Maternit...|
|483569726| 48.82353579999984|2.2768316999999985|{website -> https...|
|670633220|48.722277100000056|2.4525995000000083|{name -> Centre H...|
|783760856| 48.83526160000006|         2.2442898|{name -> Centre d...|
+---------+------------------+------------------+--------------------+
only showing top 5 rows



In [33]:
show_row_details(670633220)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id        | 670633220                                                                                                                                                                                                                                                                                          

In [35]:
doctor_df.count()

1298

In [36]:
doctor_df.show(5)

+---------+------------------+------------------+--------------------+
|       id|          latitude|         longitude|                tags|
+---------+------------------+------------------+--------------------+
|293986528| 48.86378410000007|2.3814177000000036|{amenity -> docto...|
|302305751|        48.5833366|2.2414200000000104|{amenity -> docto...|
|416708946|48.872033599999774|2.3765707000000202|{website -> https...|
|456140610| 48.75835359999986| 3.048538999999998|{amenity -> docto...|
|477197148|49.010437500000016|2.0296950999999948|{amenity -> doctors}|
+---------+------------------+------------------+--------------------+
only showing top 5 rows



In [38]:
show_row_details(416708946)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id        | 416708946                                                                                                                                                                                                                                                
 type      | 0                                                                                                                                                                                                                                                        
 latitude  | 48.872033599999774                                                                                                                                                                                    

In [89]:
from pyspark.sql import DataFrame


def geo_df_convertor(source_df: DataFrame, lat_col_name: str, long_col_name: str, source_epsg_code: str,
                     target_epsg_code: str, target_geo_col_name: str = "location"):
    """
    This function takes a dataframe with gps coordinate column(string type), convert the string column to a geometry column. The returned dataframe can be stored as geoparquet.
    :param source_df: 
    :type source_df: 
    :param long_col_name: longitude column name
    :type long_col_name: str
    :param lat_col_name: latitude column name
    :type lat_col_name: str
    :param source_epsg_code: the csr code of the input gps coordinates
    :type source_epsg_code: str
    :param target_epsg_code: the csr code of the output gps coordinates
    :type target_epsg_code: str
    :param target_geo_col_name: 
    :type target_geo_col_name: 
    :return: 
    :rtype: 
    """
    # create a temp view of the source df, the table name is the name of the data frame
    temp_table_name = "raw_source_df"
    source_df.createOrReplaceTempView(f"{temp_table_name}")
    target_df = sedona.sql(f"""
    SELECT id, tags,
    ST_Transform(ST_Point(CAST({long_col_name} AS Decimal(24,20)), CAST({lat_col_name} AS Decimal(24,20))), '{source_epsg_code}', '{target_epsg_code}') AS {target_geo_col_name} from {temp_table_name}""")
    return target_df


def geo_table_convertor(source_table_name: str, lat_col_name: str, long_col_name: str, source_epsg_code: str,
                        target_epsg_code: str, target_geo_col_name: str = "location"):
    """
    This function takes a spark temp view with gps coordinate column(string type), convert the string column to a geometry column. The returned dataframe can be stored as geoparquet.
    :param source_table_name: 
    :type source_table_name: 
    :param target_geo_col_name: 
    :type target_geo_col_name: 
    :param long_col_name: 
    :type long_col_name: 
    :param lat_col_name: 
    :type lat_col_name: 
    :param source_epsg_code: 
    :type source_epsg_code: 
    :param target_epsg_code: 
    :type target_epsg_code: 
    :return: 
    :rtype: 
    """
    # create a temp view of the source df, the table name is the name of the data frame
    target_df = sedona.sql(f"""
    SELECT 
    ST_Transform(ST_Point(CAST({long_col_name} AS Decimal(24,20)), CAST({lat_col_name} AS Decimal(24,20))), '{source_epsg_code}', '{target_epsg_code}') AS {target_geo_col_name} from {source_table_name}""")
    return target_df


In [90]:
# Set up the epsg code value, osm uses epsg:25832
source_epsg_code = "epsg:4326"
# eu centered epsg code, more information can be found https://epsg.io/25832
target_epsg_code = "epsg:4326"

hospital_geo_table_name = "hospital_geo"
hospital_geo_df = geo_df_convertor(hospital_df,"latitude","longitude",source_epsg_code,target_epsg_code)
hospital_geo_df.cache()
hospital_geo_df.createOrReplaceTempView(hospital_geo_table_name)

In [91]:
hospital_geo_df.show()

+----------+--------------------+--------------------+
|        id|                tags|            location|
+----------+--------------------+--------------------+
| 452401907|{amenity -> hospi...|POINT (2.29171149...|
| 476313165|{name -> Maternit...|POINT (2.41460189...|
| 483569726|{website -> https...|POINT (2.27683169...|
| 670633220|{name -> Centre H...|POINT (2.45259950...|
| 783760856|{name -> Centre d...|POINT (2.2442898 ...|
| 977194624|{healthcare:speci...|POINT (2.35491029...|
|1224158095|{name -> Centre d...|POINT (2.24444949...|
|1225227409|{amenity -> clini...|POINT (2.26147650...|
|1362787029|{amenity -> clini...|POINT (2.35780950...|
|1462112612|{website -> https...|POINT (2.2777715 ...|
|1684818336|{name -> Institut...|POINT (2.34351969...|
|1685650816|{website -> https...|POINT (2.33943379...|
|1744066136|{name -> Clinique...|POINT (2.87306660...|
|1763282456|{name -> Clinique...|POINT (2.26171229...|
|1768419851|{name -> CMS et P...|POINT (2.34639360...|
|194594113

In [110]:
def get_near_hospital(patient_loc:str, distance:float):
    """
    This function get the nearest hospital based on distance with a given patient location
    :param patient_loc: gps coordinates in format "POINT(longitude, latitude)"
    :param distance: the max distance between hospital and patient
    :return: 
    """
    tmp_df = sedona.sql(f"""
                         SELECT 
                         hospital_geo.id, 
                         hospital_geo.tags,
                         ST_AsGeoJSON(ST_Transform(hospital_geo.location, '{target_epsg_code}', 'epsg:4326')) hospital_point, 
                         ST_DistanceSphere(ST_GeomFromWKT('{patient_loc}'), hospital_geo.location) distance_meter
                         FROM hospital_geo 
""")
    near_hospital_df= tmp_df.filter(tmp_df.distance_meter<=distance)
    return near_hospital_df

In [182]:
casd = "POINT(2.3081911 48.8190155)"
kb = "POINT(2.3573828 48.8121344)"
resu_df= get_near_hospital(kb, 5000)
resu_df.show()

+----------+--------------------+--------------------+------------------+
|        id|                tags|      hospital_point|    distance_meter|
+----------+--------------------+--------------------+------------------+
| 977194624|{healthcare:speci...|{"type":"Point","...|3498.0786372043644|
|1362787029|{amenity -> clini...|{"type":"Point","...|1254.8694619275752|
|1684818336|{name -> Institut...|{"type":"Point","...|3541.1455120259116|
|1685650816|{website -> https...|{"type":"Point","...|3789.4252637925338|
|2495910498|{healthcare:speci...|{"type":"Point","...|2228.9702311958245|
|2506232459|{website -> http:...|{"type":"Point","...|3050.1027677005413|
|3624753425|{website -> https...|{"type":"Point","...| 4880.471899182354|
|4416374303|{name -> Clinique...|{"type":"Point","...|  2897.81069807685|
|4657177274|{amenity -> clini...|{"type":"Point","...| 2209.855315303411|
|4751279276|{amenity -> clini...|{"type":"Point","...|3276.3427584854035|
|4936398378|{amenity -> clini...|{"typ

In [183]:
kepler_map_path = f"{data_dir}/tmp/near_hospital_map.html"

sedona_kepler_map = SedonaKepler.create_map(df=resu_df,name="near_hospital")

sedona_kepler_map.save_to_html(file_name=kepler_map_path)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to C:\Users\PLIU\Documents\git\GeoParquetAndSedona/data/tmp/near_hospital_map.html!


In [118]:
resu_df.count()
resu_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- tags: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- hospital_point: string (nullable = true)
 |-- distance_meter: double (nullable = true)



In [82]:
show_row_details(2506232459)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id        | 2506232459                                                                                                                                                                      
 type      | 0                                                                                                                                                                               
 latitude  | 48.81152669999993                                                                                                                                                               
 longitude | 2.3157396000000015                                                                                                                                                              
 nodes     | []                                   

## get hospital count in the below commune

In [147]:
from sedona.spark import *
paris_polygon_df= fr_commune_df.filter(col("name")=="paris").withColumn("centroid",ST_Centroid(col("geometry")))
paris_polygon_df.show()
paris_polygon_df.createOrReplaceTempView("paris_polygon")


+--------------------+-----+-----+--------------------+
|            geometry|insee| name|            centroid|
+--------------------+-----+-----+--------------------+
|POLYGON ((2.22412...|75056|paris|POINT (2.34287643...|
+--------------------+-----+-----+--------------------+



In [175]:
distance = 8000
query = f"SELECT fr.name FROM paris_polygon p, fr_commune fr WHERE ST_DWithin(p.centroid, fr.geometry,{distance},true)"
filtered_commune_df = sedona.sql(query)

In [176]:
print(f"total commune count: {filtered_commune_df.count()}")
filtered_commune_df.show()

total commune count: 30
+--------------------+
|                name|
+--------------------+
|boulogne-billancourt|
|           châtillon|
|    levallois-perret|
|              vanves|
|               paris|
|           vincennes|
|            bagnolet|
|         romainville|
|   neuilly-sur-seine|
|           les lilas|
|         saint-mandé|
|le pré-saint-gervais|
| issy-les-moulineaux|
|          courbevoie|
|           montreuil|
|       aubervilliers|
|saint-ouen-sur-seine|
|   charenton-le-pont|
|              pantin|
|      ivry-sur-seine|
+--------------------+
only showing top 20 rows



In [177]:
target_commune = [row["name"] for row in filtered_commune_df.select("name").collect()]

print(target_commune)

['boulogne-billancourt', 'châtillon', 'levallois-perret', 'vanves', 'paris', 'vincennes', 'bagnolet', 'romainville', 'neuilly-sur-seine', 'les lilas', 'saint-mandé', 'le pré-saint-gervais', 'issy-les-moulineaux', 'courbevoie', 'montreuil', 'aubervilliers', 'saint-ouen-sur-seine', 'charenton-le-pont', 'pantin', 'ivry-sur-seine', 'asnières-sur-seine', 'gentilly', 'le kremlin-bicêtre', 'arcueil', 'villejuif', 'cachan', 'malakoff', 'montrouge', 'clichy-la-garenne', 'bagneux']


In [178]:
target_commune_df = fr_commune_df.filter(col("name").isin(target_commune))
target_commune_df.show()
target_commune_df.createOrReplaceTempView("target_commune")

+--------------------+-----+--------------------+
|            geometry|insee|                name|
+--------------------+-----+--------------------+
|POLYGON ((3.26578...|02043|             bagneux|
|POLYGON ((3.08236...|03069|           châtillon|
|POLYGON ((3.79568...|51032|             bagneux|
|POLYGON ((5.69078...|39122|           châtillon|
|POLYGON ((5.84683...|54041|             bagneux|
|POLYGON ((4.62395...|69050|           châtillon|
|POLYGON ((3.16833...|03015|             bagneux|
|POLYGON ((2.22279...|92012|boulogne-billancourt|
|POLYGON ((1.75451...|62588|           montreuil|
|POLYGON ((1.71683...|36011|             bagneux|
|POLYGON ((1.35566...|28267|           montreuil|
|POLYGON ((-0.8686...|85148|           montreuil|
|POLYGON ((2.27139...|92020|           châtillon|
|POLYGON ((2.27102...|92044|    levallois-perret|
|POLYGON ((2.27261...|92075|              vanves|
|POLYGON ((2.22412...|75056|               paris|
|POLYGON ((2.41844...|94080|           vincennes|


In [179]:
commune_hospital_df = sedona.sql(f"""
 select 
 target_commune.name, target_commune.geometry, hospital_geo.id
 FROM target_commune, hospital_geo 
 WHERE 
 ST_Contains(target_commune.geometry, hospital_geo.location)
""")

commune_hospital_df.show()
commune_hospital_df.createOrReplaceTempView("commune_hospital")

+--------------------+--------------------+-----------+
|                name|            geometry|         id|
+--------------------+--------------------+-----------+
|boulogne-billancourt|POLYGON ((2.22279...| 3189803775|
|boulogne-billancourt|POLYGON ((2.22279...|  783760856|
|boulogne-billancourt|POLYGON ((2.22279...| 1224158095|
|    levallois-perret|POLYGON ((2.27102...| 5719766166|
|               paris|POLYGON ((2.22412...|10883369653|
|               paris|POLYGON ((2.22412...| 4936398378|
|               paris|POLYGON ((2.22412...| 7923610571|
|               paris|POLYGON ((2.22412...| 9073838043|
|               paris|POLYGON ((2.22412...|10061970658|
|               paris|POLYGON ((2.22412...| 8269359942|
|               paris|POLYGON ((2.22412...| 1362787029|
|               paris|POLYGON ((2.22412...| 9247842736|
|               paris|POLYGON ((2.22412...| 4657177274|
|               paris|POLYGON ((2.22412...| 3624753425|
|               paris|POLYGON ((2.22412...| 7568

In [180]:
hospital_count_df = sedona.sql("SELECT c.name, c.geometry, count(*) as hospital_count FROM commune_hospital c GROUP BY c.name, c.name, c.geometry sort by hospital_count desc")

In [181]:
kepler_map_path = f"{data_dir}/tmp/hospital_count_map.html"

sedona_kepler_map = SedonaKepler.create_map(df=hospital_count_df,name="hospital_count_df")

sedona_kepler_map.save_to_html(file_name=kepler_map_path)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to C:\Users\PLIU\Documents\git\GeoParquetAndSedona/data/tmp/hospital_count_map.html!
